In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
# 1. Leia o arquivo 'videos-preparados.snappy.parquet' no dataframe 'df_video'
df_video = spark.read.parquet('drive/MyDrive/Colab Notebooks/colab_ebac/m29_spark_data/videos-preparados.snappy.parquet')
df_video.show(5)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|mukbang|378858|   18860|17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|   news|  6379|    4853|  808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|
|How Biden&#39;s s...|--ixiTypG8g|  2022-08-24|   news|  1029|    2347|   97434|     100810|202

In [15]:
# 2. Calcule a quantidade de registros para cada valor único da coluna "Keyword"
df_video.groupBy('Keyword').agg(
    count_distinct('Video ID').alias('Contagem_unicos')
    ).orderBy(desc('Contagem_unicos')).show()

+----------------+---------------+
|         Keyword|Contagem_unicos|
+----------------+---------------+
|             cnn|             50|
|       interview|             50|
|          crypto|             50|
|    data science|             50|
|        trolling|             50|
|        tutorial|             50|
|          marvel|             50|
|game development|             50|
|         mrbeast|             50|
|         physics|             50|
|             sat|             49|
|         history|             49|
|           cubes|             49|
|        reaction|             49|
|          sports|             49|
|            asmr|             49|
|computer science|             48|
|            food|             48|
|          how-to|             48|
|machine learning|             48|
+----------------+---------------+
only showing top 20 rows



In [27]:
# 3. Calcule a média da coluna "Interaction" para cada valor único da coluna 'Keyword'
df_video.groupBy('Keyword').agg(
    format_number(avg('Interaction'), 0).alias('Media_interaction')
).show()

+----------------+-----------------+
|         Keyword|Media_interaction|
+----------------+-----------------+
|computer science|        1,226,793|
|            lofi|        4,167,086|
|         finance|          708,543|
|             cnn|          570,651|
|           apple|       10,873,628|
|            news|          251,689|
|         mukbang|       11,053,630|
|       education|        2,750,839|
|       interview|        3,044,867|
|          crypto|          413,676|
|   mathchemistry|        3,427,343|
|            food|        5,352,944|
|    data science|          562,465|
|        trolling|        1,484,585|
|        tutorial|        6,936,688|
|      literature|          881,726|
|             sat|        1,098,927|
|         history|       15,652,693|
|           cubes|       15,043,961|
|           music|       29,691,370|
+----------------+-----------------+
only showing top 20 rows



In [26]:
# 4. Calcule o valor máximo da coluna "Interaction" para cada valor único da coluna 'Keyword' e nomeie de 'Rank Interactions',
# em seguida ordene pela nova coluna em ordem decrescente

df_video.groupBy('Keyword').agg(
    max('Interaction').alias('Rank Interactions')
).orderBy(desc('Rank Interactions')).show()

+--------+-----------------+
| Keyword|Rank Interactions|
+--------+-----------------+
| animals|       1593623628|
|   music|        922551152|
|     bed|        532691631|
| history|        440187490|
|   apple|        429916936|
| mrbeast|        300397699|
|  google|        239385460|
|business|        210025196|
|   cubes|        170925917|
|  sports|        106924567|
| mukbang|         87433858|
|    lofi|         86445177|
|tutorial|         69616442|
|  movies|         65253870|
|  marvel|         56247330|
|  how-to|         53053975|
|    food|         48754479|
| physics|         43463298|
|    asmr|         34411125|
|nintendo|         32268486|
+--------+-----------------+
only showing top 20 rows



In [32]:
# 5. Calcule a média e a variância da coluna 'Views' para cada valor único da coluna 'Keyword'
df_video.groupBy('Keyword').agg(
    format_number(avg('Views'), 0).alias('Media_views'),
    format_number(variance('Views'), 0).alias('Variancia_views')
).show()

+----------------+-----------+--------------------+
|         Keyword|Media_views|     Variancia_views|
+----------------+-----------+--------------------+
|computer science|  1,191,959|   2,812,198,681,651|
|            lofi|  4,089,363| 184,620,964,147,668|
|         finance|    694,223|   3,304,483,175,097|
|             cnn|    554,240|     156,342,361,847|
|           apple| 10,746,930|4,299,927,977,442...|
|            news|    247,492|     106,751,257,667|
|         mukbang| 10,904,772| 558,607,323,897,318|
|       education|  2,684,433|  18,335,722,493,392|
|       interview|  2,966,112|  18,192,209,960,343|
|          crypto|    404,608|   3,513,691,634,369|
|   mathchemistry|  3,328,125|  24,914,670,652,568|
|            food|  5,252,406|  73,263,741,281,548|
|    data science|    544,772|     547,933,652,535|
|        trolling|  1,420,141|   6,932,651,793,973|
|        tutorial|  6,761,032| 136,962,659,686,446|
|      literature|    863,021|     938,052,188,421|
|           

In [34]:
# 6. Calcule a média, o valor mínimo e o valor máximo de 'Views' para cada valor único da coluna 'Keyword', sem casas decimais
df_video.groupBy('Keyword').agg(
    format_number(avg('Views'), 0).alias('Media_views'),
    format_number(min('Views'), 0).alias('Min_views'),
    format_number(max('Views'), 0).alias('Max_views'),
).show()

+----------------+-----------+---------+-----------+
|         Keyword|Media_views|Min_views|  Max_views|
+----------------+-----------+---------+-----------+
|computer science|  1,191,959|   16,115|  7,004,107|
|            lofi|  4,089,363|    6,817| 84,747,957|
|         finance|    694,223|    1,195|  9,450,554|
|             cnn|    554,240|   51,269|  1,889,320|
|           apple| 10,746,930|    1,954|425,478,119|
|            news|    247,492|   10,642|  1,465,011|
|         mukbang| 10,904,772|    3,618| 86,169,225|
|       education|  2,684,433|    6,611| 17,103,736|
|       interview|  2,966,112|    2,587| 22,529,756|
|          crypto|    404,608|    1,599| 11,805,668|
|   mathchemistry|  3,328,125|       25| 18,496,859|
|            food|  5,252,406|   47,430| 48,018,833|
|    data science|    544,772|      911|  3,069,097|
|        trolling|  1,420,141|    5,388| 14,286,302|
|        tutorial|  6,761,032|   19,323| 68,512,549|
|      literature|    863,021|    2,847|  4,23

In [35]:
# 7. Mostre o primeiro e o último 'Published At' para cada valor único da coluna 'Keyword'
df_video.groupBy('Keyword').agg(
    min('Published At').alias('Primeiro_published'),
    max('Published At').alias('Ultimo_published')
).show()

+----------------+------------------+----------------+
|         Keyword|Primeiro_published|Ultimo_published|
+----------------+------------------+----------------+
|computer science|        2009-08-20|      2022-08-12|
|            lofi|        2019-12-08|      2022-08-24|
|         finance|        2012-11-27|      2022-08-24|
|             cnn|        2022-07-14|      2022-08-24|
|           apple|        2016-11-02|      2022-08-24|
|            news|        2022-08-18|      2022-08-24|
|         mukbang|        2020-02-29|      2022-08-24|
|       education|        2008-07-25|      2022-08-24|
|       interview|        2016-01-05|      2022-08-24|
|          crypto|        2022-03-11|      2022-08-24|
|   mathchemistry|        2013-04-15|      2022-05-03|
|            food|        2017-05-31|      2022-08-24|
|    data science|        2018-06-23|      2022-08-24|
|        trolling|        2020-06-14|      2022-08-24|
|        tutorial|        2017-02-01|      2022-08-23|
|      lit

In [37]:
# 8. Conte todos os 'title' de forma normal e todos os únicos e verifique se há diferença
count_title = df_video.select('title').count()
count_title_unico = df_video.select('title').distinct().count()
print(f'QTD title: {count_title}')
print(f'QTD title unico: {count_title_unico}')

QTD title: 1869
QTD title unico: 1854


In [38]:
# 9. Mostre a quantidade de registros ordenados por ano em ordem ascendente
df_video.groupBy(year('Published At').alias('Ano')).count().orderBy('Ano').show()


+----+-----+
| Ano|count|
+----+-----+
|2007|    2|
|2008|    1|
|2009|    9|
|2010|    6|
|2011|    4|
|2012|   12|
|2013|    6|
|2014|   10|
|2015|   15|
|2016|   34|
|2017|   47|
|2018|   57|
|2019|   86|
|2020|  158|
|2021|  229|
|2022| 1193|
+----+-----+



In [40]:
# 10. Mostre a quantidade de registros ordenados por ano e mês em ordem ascendente
df_video.groupBy(year('Published At').alias('Ano'), month('Published At').alias('Mes'))\
    .count().orderBy('Ano', 'Mes').show()

+----+---+-----+
| Ano|Mes|count|
+----+---+-----+
|2007|  7|    1|
|2007| 12|    1|
|2008|  7|    1|
|2009|  2|    2|
|2009|  6|    2|
|2009|  7|    1|
|2009|  8|    1|
|2009| 10|    1|
|2009| 12|    2|
|2010|  3|    1|
|2010|  5|    2|
|2010|  6|    1|
|2010|  9|    1|
|2010| 10|    1|
|2011|  2|    1|
|2011|  5|    1|
|2011|  9|    1|
|2011| 10|    1|
|2012|  1|    1|
|2012|  2|    3|
+----+---+-----+
only showing top 20 rows



In [48]:
# 11. Calcule a média acumulativa de ‘Likes’ para cada ‘Keyword’ ao longo dos anos.

janela_acumulativa = Window.partitionBy('Keyword').orderBy('Ano').rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_likes_med_acum = df_video.withColumn('Ano', year('Published At'))

df_likes_med_acum = df_likes_med_acum.withColumn(
    'Med_Likes_acumulado',
    avg('Likes').over(janela_acumulativa))

df_likes_med_acum.groupBy('Keyword', 'Ano').agg(
    avg('Med_Likes_acumulado').alias('Med_Likes_acumulado')
).show()

+-------+----+-------------------+
|Keyword| Ano|Med_Likes_acumulado|
+-------+----+-------------------+
|animals|2009|          1357197.0|
|animals|2010|           650321.0|
|animals|2013|         3197276.75|
|animals|2014| 3010213.1166666667|
|animals|2019| 2950868.5714285714|
|animals|2020|  1880639.258017754|
|animals|2021|  1388437.658048009|
|animals|2022|  923691.7527111218|
|  apple|2016|          4144389.0|
|  apple|2021|          2091325.0|
|  apple|2022|  305566.5841958186|
|   asmr|2020|           148120.0|
|   asmr|2021|  339133.0833333333|
|   asmr|2022|  79013.54289805984|
|    bed|2007|          416104.75|
|    bed|2009|  710739.8944444446|
|    bed|2010|           573747.5|
|    bed|2011|  582662.7767857143|
|    bed|2017|  520252.3087179487|
|    bed|2018|  478807.3155112044|
+-------+----+-------------------+
only showing top 20 rows

